In [1]:
import fvdb
from fvdb.nn import VDBTensor
import torch
import fvdb.nn as fvnn

In [2]:
class DownSampleBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = fvdb.nn.SparseConv3d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, bias=False)
        self.norm1 = fvdb.nn.BatchNorm(num_features=out_channels)

        self.conv2 = fvdb.nn.SparseConv3d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, bias=False)
        self.norm2 = fvdb.nn.BatchNorm(num_features=out_channels)

        self.relu = fvdb.nn.ReLU(inplace=True)
        self.maxpool = fvdb.nn.MaxPool(2, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.norm2(x)

        return x, self.maxpool(x)
    
class UpSampleBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.upconv = fvdb.nn.SparseConv3d(in_channels=in_channels, out_channels=out_channels, transposed=True)
        self.upNorm = fvdb.nn.BatchNorm(out_channels)

        # jcat will double the channel size
        self.conv1 = fvdb.nn.SparseConv3d(in_channels=out_channels*2, out_channels=out_channels, kernel_size=3, stride=1, bias=False)
        self.norm1 = fvdb.nn.BatchNorm(out_channels)

        self.conv2 = fvdb.nn.SparseConv3d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, bias=False)
        self.norm2 = fvdb.nn.BatchNorm(out_channels)

    def forward(self, x, x_skip):
        x = self.upconv(x, x_skip.grid)
        x = self.upNorm(x)
        x = fvdb.jcat([x, x_skip], dim=1)

        x = self.conv1(x)
        x = self.norm1(x)

        x = self.conv2(x)
        x = self.norm2(x)

        return x

In [3]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.NUM_CHANNELS = (1, 8, 8, 16, 16, 32, 32, 16, 16, 8, 8, 1)

        self.downSampleBlock1 = DownSampleBlock(1, 8)
        self.downSampleBlock2 = DownSampleBlock(8, 16)
        self.downSampleBlock3 = DownSampleBlock(16, 32)
        self.downSampleBlock4 = DownSampleBlock(32, 64)

        self.bottleneck1 = fvdb.nn.SparseConv3d(in_channels=64, out_channels=86, kernel_size=3, stride=1, bias=False)
        self.bottleNorm1 = fvdb.nn.BatchNorm(num_features=86)
        self.bottleneck2 = fvdb.nn.SparseConv3d(in_channels=86, out_channels=86, kernel_size=3, stride=1, bias=False)
        self.bottleNorm2 = fvdb.nn.BatchNorm(86)

        self.upsample4 = UpSampleBlock(86, 64)
        self.upsample3 = UpSampleBlock(64, 32)
        self.upsample2 = UpSampleBlock(32, 16)
        self.upsample1 = UpSampleBlock(16, 8)

    def forward(self, x):
        og1, out1 = self.downSampleBlock1(x)
        og2, out2 = self.downSampleBlock2(out1)
        og3, out3 = self.downSampleBlock3(out2)
        og4, out4 = self.downSampleBlock4(out3)

        out = self.bottleneck1(out4)
        out = self.bottleNorm1(out)
        out = self.bottleneck2(out)
        out = self.bottleNorm2(out)

        out = self.upsample4(out, og4)
        out = self.upsample3(out, og3)
        out = self.upsample2(out, og2)
        out = self.upsample1(out, og1)

        return out

In [4]:
grid_batch, labels, names = fvdb.load("./data/training_data/regions/3.0.0.nvdb", device='cuda')
labels = labels.to(torch.float)
print("Loaded grid batch total number of voxels: ", grid_batch.total_voxels)
print("Loaded grid batch data type: %s, device: %s" % (labels.dtype, labels.device))

Loaded grid batch total number of voxels:  8218378
Loaded grid batch data type: torch.float32, device: cuda:0


In [5]:
active_feature_list = torch.ones(grid_batch[0].total_enabled_voxels, device='cuda', dtype=torch.float32).unsqueeze(-1)
active_feature  = fvdb.JaggedTensor(active_feature_list).to(torch.float32)

In [6]:
sinput = fvdb.nn.VDBTensor(grid_batch, active_feature)
model = Model().to('cuda')

In [7]:
soutput = model(sinput)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 5.68 GiB of which 16.88 MiB is free. Including non-PyTorch memory, this process has 5.64 GiB memory in use. Of the allocated memory 5.53 GiB is allocated by PyTorch, and 22.20 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)